In [1]:
using Plots
using Distributions
gr()

# アヒル本4章のデータ
x=Real[ 24, 24, 26, 32, 33, 35, 38, 40, 40, 43, 43, 44, 48, 52,  56,  56,  57,  58,  59,  59]
y=Real[472,403,454,575,546,781,750,601,814,792,745,837,868,988,1092,1007,1233,1202,1123,1314]

# Plot
plot(x,y,seriestype=:scatter,title="Chap4-5 Salary Data")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 1200 
 
 
 Chap4-5 Salary Data 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [2]:
function likehood(a, b, sigma)
    y_pred = a + b*x
    likehoods = [logpdf(Normal(aa, sigma), bb) for (aa,bb) in zip(y_pred,y)]
    l_sum = sum(likehoods)
    return l_sum
end

function prior(a, b, sigma)
    a_prior     = logpdf(Normal(0, 100),a)
    b_prior     = logpdf(Normal(0, 100),b)
    sigma_prior = logpdf(Uniform(0,1000),sigma)
    return a_prior + b_prior + sigma_prior
end

function posterior(a, b, sigma)
    return(likehood(a, b, sigma) + prior(a, b, sigma))
end

function proposal(a, b, sigma)
    return [rand(Normal(mu, sd)) for (mu, sd) in zip([a, b, sigma],[0.1, 0.1, 0.1])]
end

proposal (generic function with 1 method)

In [10]:
#MH法
# a, b, sigmaの初期値; Chain 1つ分
Init = Real[-100, 1, 1] 
Iteration = 200000
BurnIn    =  90000
chain = Array[Init]

for i in 1:Iteration
    proposal_draw   = proposal(chain[i][1], chain[i][2], chain[i][3])
    
    tmp1 = posterior(proposal_draw[1],proposal_draw[2],proposal_draw[3])
    tmp2 = posterior(chain[i][1], chain[i][2], chain[i][3])
    alpha = min(1,exp(tmp1-tmp2))
    
    u = rand(Uniform(0,1))

    if u < alpha
        push!(chain,proposal_draw)
    else
        push!(chain,chain[i])
    end
end

# BurnInは除外
chain = chain[BurnIn+1:end]

# 結果の取り出し
a = [chain[i][1] for i in 1:length(chain)]
b = [chain[i][2] for i in 1:length(chain)]
sigma = [chain[i][3] for i in 1:length(chain)]

println("MCMC complete")

MCMC complete


In [11]:
mean(a),mean(b),mean(sigma)

(-70.13509646340123, 20.862066462389677, 85.25946648157439)